!kaggle competitions download -c home-data-for-ml-course

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
train = pd.DataFrame(pd.read_csv('train.csv'))
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
test = pd.DataFrame(pd.read_csv('test.csv'))
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
types = {i: train[i].dtype for i in train.columns}
types

{'Id': dtype('int64'),
 'MSSubClass': dtype('int64'),
 'MSZoning': dtype('O'),
 'LotFrontage': dtype('float64'),
 'LotArea': dtype('int64'),
 'Street': dtype('O'),
 'Alley': dtype('O'),
 'LotShape': dtype('O'),
 'LandContour': dtype('O'),
 'Utilities': dtype('O'),
 'LotConfig': dtype('O'),
 'LandSlope': dtype('O'),
 'Neighborhood': dtype('O'),
 'Condition1': dtype('O'),
 'Condition2': dtype('O'),
 'BldgType': dtype('O'),
 'HouseStyle': dtype('O'),
 'OverallQual': dtype('int64'),
 'OverallCond': dtype('int64'),
 'YearBuilt': dtype('int64'),
 'YearRemodAdd': dtype('int64'),
 'RoofStyle': dtype('O'),
 'RoofMatl': dtype('O'),
 'Exterior1st': dtype('O'),
 'Exterior2nd': dtype('O'),
 'MasVnrType': dtype('O'),
 'MasVnrArea': dtype('float64'),
 'ExterQual': dtype('O'),
 'ExterCond': dtype('O'),
 'Foundation': dtype('O'),
 'BsmtQual': dtype('O'),
 'BsmtCond': dtype('O'),
 'BsmtExposure': dtype('O'),
 'BsmtFinType1': dtype('O'),
 'BsmtFinSF1': dtype('int64'),
 'BsmtFinType2': dtype('O'),
 'BsmtF

In [5]:
int_float = [i for i in train.columns
             if train[i].dtype in ['int64', 'float64']]
del int_float[-1], int_float[0]

obj = [i for i in train.columns
             if train[i].dtype == 'object']

In [6]:
train[obj] = train[obj].fillna('missed')
test[obj] = test[obj].fillna('missed')
train[int_float] = train[int_float].fillna(0)
test[int_float] = test[int_float].fillna(0)

In [7]:
train.isnull().sum().nunique()

1

In [8]:
test.isnull().sum().nunique()

1

In [9]:
train_dict = {i: train[i].nunique() for i in train.columns}
test_dict = {i: test[i].nunique() for i in test.columns}

In [19]:
not_common = []
for i, j in zip(train_dict, test_dict):
    if train_dict[i] != test_dict[j]:
        not_common.append(i)
len(train.columns.tolist())

81

train[obj].fillna('missed', inplace=True)
test[obj].fillna('missed', inplace=True)
train[int_float].fillna(0, inplace=True)
test[int_float].fillna(0, inplace=True)

In [16]:
enc = OrdinalEncoder()
train_enc = enc.fit_transform(train.drop(not_common, axis=1))
test_enc = enc.transform(test.drop(not_common, axis=1))

KeyError: "['SalePrice'] not found in axis"

In [17]:
X = train_enc.drop(['SalePrice', 'Id'], axis=1)
y = train_enc.SalePrice
np.unique(X.dtypes)

array([dtype('int64'), dtype('float64')], dtype=object)

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [24]:
model = XGBRegressor(random_state=1, learning_rate=0.01, n_estimators=40000, n_jobs=4)
model.fit(x_train, y_train,
         early_stopping_rounds=20, 
         eval_set=[(x_test, y_test)],
         verbose=False)
pred = model.predict(train_enc.drop(['SalePrice', 'Id'], axis=1))

In [25]:
mean_absolute_error(pred, y)

9602.316556078767

In [27]:
test_enc['PoolQC_Fa', 'RoofMatl_ClyTile',
         'Heating_Floor', 'RoofMatl_Metal',
         'Exterior2nd_Other', 'Heating_OthW',
         'Exterior1st_ImStucc', 'MiscFeature_TenC',
         'HouseStyle_2.5Fin', 'RoofMatl_Roll',
         'Exterior1st_Stone', 'GarageQual_Ex',
         'Condition2_RRAe', 'RoofMatl_Membran',
         'Utilities_NoSeWa', 'Condition2_RRAn',
         'Condition2_RRNn', 'Electrical_Mix'] = 0
fin_pred = model.predict(test_enc)

ValueError: feature_names mismatch: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_AllPub', 'Utilities_NoSeWa', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev',

In [28]:
output = pd.DataFrame(
    {'Id': test.Id,
     'SalePrice': fin_pred}).to_csv('submission.csv', index=False)

NameError: name 'fin_pred' is not defined

In [78]:
!kaggle competitions submit -c home-data-for-ml-course -f submission.csv -m "Message"

100%|██████████████████████████████████████| 21.2k/21.2k [00:05<00:00, 4.00kB/s]
Successfully submitted to Housing Prices Competition for Kaggle Learn Users